# Actividad 4 | Métricas de calidad de resultados


Al finalizar esta actividad, habrás aprendido a aplicar diferentes estrategias para medir bajo diferentes métricas, la calidad de modelos de aprendizaje automático (supervisados o no supervisados) aplicados al procesamiento de grandes volúmenes de datos (Big Data), usando la biblioteca PySpark. Para esta actividad, se estará trabajando con la muestra representativa M de la población P del problema de investigación que decidiste seleccionar desde el inicio del curso, además de retomar los criterios de particionamiento definidos en la actividad 3 del Módulo 4.


In [ ]:
import functools
import multiprocessing

import pyspark.sql.functions as F


from pyspark.sql import SparkSession, DataFrame
from pyspark.sql.types import (
    StructType,
    StructField,
    DateType,
    IntegerType,
    StringType,
    DoubleType,
    BooleanType,
)

from pyspark.ml import Pipeline
from pyspark.ml.tuning import TrainValidationSplit, ParamGridBuilder
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.regression import (
    LinearRegression,
    GBTRegressor,
)
from pyspark.ml.feature import (
    StandardScaler,
    StringIndexer,
    OneHotEncoder,
    VectorAssembler,
    QuantileDiscretizer,
)

In [2]:
# How much memory from the available in the machine I'm able to use
EXECUTOR_MEMORY = "100G"
DRIVER_MEMORY = "20G"

In [3]:
spark = (
    SparkSession.builder.appName("flights")
    .config("spark.executor.memory", EXECUTOR_MEMORY)
    .config("spark.driver.memory", DRIVER_MEMORY)
    .getOrCreate()
)

spark.sparkContext.setLogLevel("ERROR")
spark.sparkContext.setCheckpointDir("./data/checkpoints")

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/06/07 14:40:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
# This should be changed according to the environment
filepath = "./data/flights"

In [5]:
filenames = [f"Combined_Flights_{y}.csv" for y in range(2018, 2019)]

### Carga de Archivos


In [6]:
schema = StructType(
    [
        StructField("FlightDate", DateType(), True),
        StructField("Airline", StringType(), True),
        StructField("Origin", StringType(), True),
        StructField("Dest", StringType(), True),
        StructField("Cancelled", BooleanType(), True),
        StructField("Diverted", StringType(), True),
        StructField("CRSDepTime", IntegerType(), True),
        StructField("DepTime", DoubleType(), True),
        StructField("DepDelayMinutes", DoubleType(), True),
        StructField("DepDelay", DoubleType(), True),
        StructField("ArrTime", DoubleType(), True),
        StructField("ArrDelayMinutes", DoubleType(), True),
        StructField("AirTime", DoubleType(), True),
        StructField("CRSElapsedTime", DoubleType(), True),
        StructField("ActualElapsedTime", DoubleType(), True),
        StructField("Distance", DoubleType(), True),
        StructField("Year", IntegerType(), True),
        StructField("Quarter", IntegerType(), True),
        StructField("Month", IntegerType(), True),
        StructField("DayofMonth", IntegerType(), True),
        StructField("DayOfWeek", IntegerType(), True),
        StructField("Marketing_Airline_Network", StringType(), True),
        StructField("Operated_or_Branded_Code_Share_Partners", StringType(), True),
        StructField("DOT_ID_Marketing_Airline", StringType(), True),
        StructField("IATA_Code_Marketing_Airline", StringType(), True),
        StructField("Flight_Number_Marketing_Airline", StringType(), True),
        StructField("Operating_Airline", StringType(), True),
        StructField("DOT_ID_Operating_Airline", StringType(), True),
        StructField("IATA_Code_Operating_Airline", StringType(), True),
        StructField("Tail_Number", StringType(), True),
        StructField("Flight_Number_Operating_Airline", StringType(), True),
        StructField("OriginAirportID", StringType(), True),
        StructField("OriginAirportSeqID", StringType(), True),
        StructField("OriginCityMarketID", StringType(), True),
        StructField("OriginCityName", StringType(), True),
        StructField("OriginState", StringType(), True),
        StructField("OriginStateFips", StringType(), True),
        StructField("OriginStateName", StringType(), True),
        StructField("OriginWac", StringType(), True),
        StructField("DestAirportID", StringType(), True),
        StructField("DestAirportSeqID", StringType(), True),
        StructField("DestCityMarketID", StringType(), True),
        StructField("DestCityName", StringType(), True),
        StructField("DestState", StringType(), True),
        StructField("DestStateFips", StringType(), True),
        StructField("DestStateName", StringType(), True),
        StructField("DestWac", StringType(), True),
        StructField("DepDel15", StringType(), True),
        StructField("DepartureDelayGroups", StringType(), True),
        StructField("DepTimeBlk", StringType(), True),
        StructField("TaxiOut", DoubleType(), True),
        StructField("WheelsOff", DoubleType(), True),
        StructField("WheelsOn", DoubleType(), True),
        StructField("TaxiIn", DoubleType(), True),
        StructField("CRSArrTime", IntegerType(), True),
        StructField("ArrDelay", DoubleType(), True),
        StructField("ArrDel15", StringType(), True),
        StructField("ArrivalDelayGroups", StringType(), True),
        StructField("ArrTimeBlk", StringType(), True),
        StructField("DistanceGroup", StringType(), True),
        StructField("DivAirportLandings", StringType(), True),
    ]
)

In [7]:
def read_files_from(path: str, files: list[str], schema) -> DataFrame:
    df = spark.createDataFrame([], schema=schema)
    dataframes = [
        spark.read.csv(f"{filepath}/{filename}", header=True, schema=schema)
        for filename in files
    ]
    return functools.reduce(DataFrame.unionAll, dataframes, df)

In [8]:
df = read_files_from(filepath, filenames, schema)

### Limpieza de Datos


In [9]:
# number of original rows
df.count()

5689512

In [10]:
# drop rows with null values
clean_df = df.dropna()

# drop columns with null values
clean_df = clean_df.na.drop()

# drop duplicated rows
clean_df = clean_df.dropDuplicates().coalesce(numPartitions=multiprocessing.cpu_count()).checkpoint()

# number of rows after cleaning
clean_df.count()

5578618

Definition of categorical columns, continuous columns and label


In [11]:
label = "ArrDelay"


continuous_cols = [
    el.name
    for el in df.schema
    if isinstance(el.dataType, (DoubleType, IntegerType))
    and el.name not in ["Year", "Quarter", "Month", "DayofMonth", "DayOfWeek"]
]

categorica_cols = [
    "Marketing_Airline_Network",
    "Operating_Airline",
    "OriginAirportID",
    "DestAirportID",
    "OriginCityName",
    "DestCityName",
    "DayOfWeek",
    "Month",
    "DistanceGroup",
]

## Construcción de la muestra M


1. Construir una muestra M que sea representativa de la población P (a partir del dataset que recolectaste desde el inicio del curso). Tomando como base el conocimiento adquirido en la Actividad 3 del Módulo 4, generarás particiones Mi de M, donde cada Mi cumple con los criterios definidos por las variables de caracterización que identificaste previamente (M será igual a la unión de todos los Mi). Para esta actividad, y a diferencia del paso previo, se deberá de tener especial cuidado para determinar el número de instancias que deberá contender cada partición Mi a generar, de tal forma que no se inyecte ningún tipo de sesgo que pueda alterar la calidad de los resultados. Como resultado, crearás una sección en el archivo Jupyter Notebook a entregar llamada “1 Construcción de la muestra M”, donde en código Python y haciendo uso de PySpark, generarás las particiones derivadas del análisis hecho. Cada paso deberá ser documentado.


In [12]:
# Calculate the kurtosis and skewness
clean_df.select(F.kurtosis(label), F.skewness(label)).show()

+------------------+------------------+
|kurtosis(ArrDelay)|skewness(ArrDelay)|
+------------------+------------------+
|143.93258992452914| 8.468000271818116|
+------------------+------------------+



La variable presenta una asimetria positiva significativa lo que indica que la distribucion de los restrasos en la llegada esta sesgada a la derecha, es decir, que hay una concentracion de vuelos con pocos o ningun retraso, pero existen algunos vuelos con retrasos extremadamente altos.

El valor de kustosis extremadamente alto sugiere que la distribucion tiene colar pesadas y picos pronunciados lo que indica un presencia muy fuerte de valores atipicos.


In [13]:
num_bins = 13

# Create a Discrete output over a continuous column
quantile_discretizer = QuantileDiscretizer(
    numBuckets=num_bins, inputCol=label, outputCol=f"{label}_discrete"
)

clean_bin_df = quantile_discretizer.fit(clean_df).transform(clean_df)

In [14]:
# Get distinct categories in a list
categories = (
    clean_bin_df.select(f"{label}_discrete")
    .distinct()
    .rdd.flatMap(lambda x: x)
    .collect()
)

In [15]:
partitions = []

# Iterate the categories and create a sample
for cat in categories:
    df_cat = clean_bin_df.filter(F.col(f"{label}_discrete") == cat)
    fraction = 0.1
    sample = (
        df_cat.drop(f"{label}_discrete")
        .sample(withReplacement=False, fraction=fraction, seed=42)
    )
    partitions.append(sample)

In [16]:
# Union all samples again
__df = spark.createDataFrame([], schema=schema)
sample_df = functools.reduce(DataFrame.unionAll, partitions, __df).coalesce(numPartitions=multiprocessing.cpu_count()).checkpoint()

In [17]:
# Number of rows in the sample
sample_df.count()

558022

In [18]:
# Calculate the kurtosis and skewness
sample_df.select(F.kurtosis(label), F.skewness(label)).show()

+------------------+------------------+
|kurtosis(ArrDelay)|skewness(ArrDelay)|
+------------------+------------------+
|142.47484681309507| 8.550897072406542|
+------------------+------------------+



## Construcción Train – Test


2. Construcción del conjunto de entrenamiento y prueba. Para este paso se asume que M = {Mi: Mi es una partición derivada de las variables de caracterización de la población} generada en el paso anterior. Para construir el conjunto de entrenamiento y prueba, se debe de calcular el porcentaje de división a usar, de tal forma que al dividir cada Mi en un conjunto de entrenamiento (Tri) y prueba (Tsi), no se inyecten sesgos que desvíen la probabilidad de ocurrencia de los patrones en cada nueva partición. Para ello, deberás de retomar la estrategia de muestreo propuesta en el paso 4 de la Actividad 3 del módulo 4. Se debe de cuidar que Tri Ç Tsi = Æ, además de que la unión de todas las particiones es igual a M. Como evidencia, se generará una nueva sección en el archivo Jupyter Notebook que construyes, llamada “2 Construcción Train – Test”, en dónde generes el código correspondiente en Python que implemente lo antes solicitado. Todo deberá de estar debidamente documentado.


In [19]:
sample_bin_df = quantile_discretizer.fit(sample_df).transform(sample_df)

In [20]:
categories = (
    sample_bin_df.select(f"{label}_discrete")
    .distinct()
    .rdd.flatMap(lambda x: x)
    .collect()
)

In [21]:
train_partitions = []
test_partitions = []

# Iterate the categories and create a sample
for cat in categories:
    # Create stratum
    stratum_df = sample_bin_df.filter(F.col(f"{label}_discrete") == cat)

    # Separate in train and test
    train_stratum_df, test_stratum_df = stratum_df.limit(10000).drop(
        f"{label}_discrete"
    ).randomSplit([0.7, 0.3])

    train_partitions.append(train_stratum_df.cache())
    test_partitions.append(test_stratum_df.cache())

In [22]:
__df = spark.createDataFrame([], schema=schema)

train_df = functools.reduce(DataFrame.unionAll, train_partitions, __df).coalesce(numPartitions=multiprocessing.cpu_count()).checkpoint()

test_df = functools.reduce(DataFrame.unionAll, test_partitions, __df).coalesce(numPartitions=multiprocessing.cpu_count()).checkpoint()

In [23]:
train_df.select(F.kurtosis(label), F.skewness(label)).show()

+------------------+------------------+
|kurtosis(ArrDelay)|skewness(ArrDelay)|
+------------------+------------------+
| 142.2852689839089| 8.677594843565547|
+------------------+------------------+



In [24]:
test_df.select(F.kurtosis(label), F.skewness(label)).show()

+------------------+------------------+
|kurtosis(ArrDelay)|skewness(ArrDelay)|
+------------------+------------------+
|130.94558909645903| 8.301698308041166|
+------------------+------------------+



## Selección de métricas para medir calidad de resultados


3. Con la finalidad de medir la calidad de resultados que se obtienen, se debe de seleccionar previamente métricas para su medición. Se recomienda que se analice a profundidad que métricas se pueden aplicar, considerando que se trabaja con grandes volúmenes de datos. Para evidenciar el análisis hecho, crearás una sección llamada “3 Selección de métricas para medir calidad de resultados”, dónde argumentarás que métricas serán usadas para medir la calidad de los modelos derivados de la etapa de entrenamiento. Dichas métricas se deberán de implementar en tu etapa de experimentación del paso 4.


**Mean Square Error**

Ventajas:

- Penaliza fuertemente los errores grandes, lo que lo hace útil cuando estos son especialmente indeseables.
- Es diferenciable, lo cual lo convierte en una función de pérdida adecuada para algoritmos de optimización basados en gradiente.

Desventajas

- Es sensible a los valores atípicos (outliers), ya que los errores se elevan al cuadrado.
- La métrica no conserva las unidades originales de la variable objetivo, lo que puede dificultar la interpretación directa de los resultados.


**Root Mean Square Error**

Ventajas:

- Está en las mismas unidades que la variable objetivo, lo que facilita la interpretación de los resultados.
- Penaliza más los errores grandes, al igual que el MSE, lo cual es útil cuando los errores grandes son más perjudiciales.

Desventajas:

- Sigue siendo sensible a los valores atípicos (outliers), ya que también se basa en el cuadrado de los errores antes de tomar la raíz.


**Mean Absolute Error**

Ventajas:

- Robusto frente a valores atípicos, ya que no eleva los errores al cuadrado.
- Fácil de interpretar, ya que representa el error promedio en las mismas unidades que la variable objetivo.

Desventajas:

- No penaliza los errores grandes tan fuertemente como el MSE o RMSE, lo que puede ser una desventaja cuando los errores grandes son especialmente críticos.


**Coeficiente de Determinacion (R^2)**

Ventajas:

- Interpretación intuitiva, ya que indica la proporción de la varianza explicada por el modelo.
- Ampliamente utilizado en regresión lineal, lo que lo hace familiar y fácil de comunicar.

Desventajas:

- No penaliza la inclusión de variables irrelevantes, lo que puede llevar a sobreajuste si se agregan demasiadas características.
- No es adecuado para modelos no lineales, ya que puede dar valores engañosos fuera del contexto de regresión lineal.


**Coeficiente de Determinacion (R^2) Ajustado**

Ventajas:

- Se ajusta según el número de predictores, lo que lo hace más adecuado que el R² tradicional para modelos de regresión múltiple.
- Útil para comparar modelos con diferente cantidad de variables, ya que penaliza la inclusión de predictores irrelevantes.

Desventajas:

- Más complejo de calcular que el R² estándar, especialmente al explicar su fórmula o interpretación.
- Menos informativo en modelos con un solo predictor, donde su valor suele ser muy similar al de R².


**Mean Absolute Percentage Error**

Ventajas:

- Interpretabilidad: Al expresarse en porcentaje, el MAPE es intuitivo y facilita la comparación del rendimiento del modelo entre diferentes conjuntos de datos.
- Medida relativa del error: Es útil cuando se trabaja con datos de distintas escalas, ya que proporciona una visión proporcional del error respecto a los valores reales.

Desventajas:

- Sensibilidad a valores reales pequeños: Si los valores reales son muy cercanos a cero, el error porcentual se amplifica de forma desproporcionada, distorsionando la evaluación.
- Indefinido para valores reales iguales a cero: Cuando el valor real es cero, el MAPE no puede calcularse, lo que limita su aplicabilidad en ciertos conjuntos de datos.


### Metricas Seleccionadas


1. Root Mean Square Error

- Preserva las unidades originales (minutos de retraso), lo que facilita su interpretación por parte de los stakeholders.
- Penaliza fuertemente los errores grandes, lo cual es adecuado para este caso, ya que los retrasos extremos (por ejemplo, superiores a 180 minutos) tienen un impacto desproporcionado en los pasajeros, las aerolíneas y las operaciones aeroportuarias.
- Amplia aceptación en la industria del transporte y la aviación para tareas de predicción de retrasos.

2. Mean Absolute Error

- Más robusta ante valores atípicos que el RMSE, lo cual es relevante dada la distribución sesgada y con colas pesadas del conjunto de datos.
- Interpretación directa y clara: indica el promedio absoluto de error en minutos, lo que permite comunicar fácilmente la precisión del modelo (por ejemplo, “en promedio, el modelo se equivoca por X minutos”).

3. Coefiente de Determinacion (R^2)

- Interpretación útil: El R² indica la proporción de la varianza en los retrasos que puede ser explicada por el modelo. Un valor cercano a 1 sugiere que el modelo captura bien la dinámica subyacente del problema; un valor cercano a 0 indica que apenas mejora respecto a una predicción basada en la media.


## Entrenamiento de Modelos de Aprendizaje


4. Construcción de modelos de aprendizaje. Para esta etapa y partiendo de la elección de algoritmos de aprendizaje (supervisado, no supervisado), aplicarás un proceso de entrenamiento que te permita construir modelos de aprendizaje que ayude a identificar los patrones de interés existentes en los datos. Se deberá de tener en claro la estrategia de entrenamiento a implementar, desde la forma en la cual se procesarán los datos hasta el ajuste de hiper – parámetros a emplear, además de los ajustes y técnicas adicionales que impidan que los modelos generados estén sobre- ajustados. Lo anterior lo deberás de aterrizar en una sección del Jupyter Notebook que se construye llamada “4 Entrenamiento de Modelos de Aprendizaje”. Se deberá documentar dicho proceso.


In [25]:
train_df = train_df.withColumnRenamed(label, "label")
test_df = test_df.withColumnRenamed(label, "label")

In [26]:
def one_hot_pipeline(input_col: str):
    indexer = StringIndexer(
        inputCol=input_col, outputCol=f"{input_col}_indexed", handleInvalid="keep"
    )
    encoder = OneHotEncoder(
        inputCol=f"{input_col}_indexed", outputCol=f"{input_col}_vec"
    )
    return Pipeline(stages=[indexer, encoder]), f"{input_col}_vec"

### Regresion Lineal

In [ ]:
def get_linear_regresion_pipeline(
    continuous: list[str], categorical: list[str], label: str
) -> TrainValidationSplit:
    one_hot_pipelines = []
    for categorical_column in categorical:
        pipe, col = one_hot_pipeline(categorical_column)
        one_hot_pipelines.append((pipe, col))

    assembled_columns = [col for col in continuous if col != label] + [
        col[1] for col in one_hot_pipelines
    ]
    assembler = VectorAssembler(inputCols=assembled_columns, outputCol="features")

    scaler = StandardScaler(
        withMean=False, withStd=True, inputCol="features", outputCol="scaled_features"
    )

    model = LinearRegression()

    pipeline = Pipeline(
        stages=[col[0] for col in one_hot_pipelines] + [assembler, scaler, model]
    )

    paramGrid = (
        ParamGridBuilder()
        .addGrid(model.regParam, [0.0, 0.01, 0.1, 1.0])
        .addGrid(model.fitIntercept, [False, True])
        .addGrid(model.elasticNetParam, [0.0, 0.5, 1.0])
        .addGrid(model.tol, [1e-4, 1e-6])
        .addGrid(model.maxIter, [10, 50])
        .build()
    )

    return TrainValidationSplit(
        estimator=pipeline,
        estimatorParamMaps=paramGrid,
        evaluator=RegressionEvaluator(),
        trainRatio=0.7,
    )

In [28]:
model = get_linear_regresion_pipeline(continuous_cols, categorica_cols, label)

In [29]:
model = model.fit(train_df)

In [30]:
model.write().overwrite().save('./models/lr')

In [31]:
prediction_df = model.transform(test_df).select("features", "label", "prediction").checkpoint()

In [32]:
evaluator_rmse = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="rmse"
)

evaluator_mae = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="mae"
)

evaluator_r2 = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="r2"
)

In [33]:
rmse = evaluator_rmse.evaluate(prediction_df)
mae = evaluator_mae.evaluate(prediction_df)
r2 = evaluator_r2.evaluate(prediction_df)

In [34]:
print(f"RMSE: {rmse}, MAE: {mae}, R^2: {r2}")

RMSE: 2.31309453379273, MAE: 1.7675790522963606, R^2: 0.997675624163221


### Gradient-Boosted Tree

In [ ]:
def get_gbt_regresion_pipeline(
    continuous: list[str], categorical: list[str], label: str
) -> TrainValidationSplit:
    one_hot_pipelines = []
    for categorical_column in categorical:
        pipe, col = one_hot_pipeline(categorical_column)
        one_hot_pipelines.append((pipe, col))

    assembled_columns = [col for col in continuous if col != label] + [
        col[1] for col in one_hot_pipelines
    ]
    assembler = VectorAssembler(inputCols=assembled_columns, outputCol="features")

    scaler = StandardScaler(
        withMean=False, withStd=True, inputCol="features", outputCol="scaled_features"
    )

    model = GBTRegressor()

    pipeline = Pipeline(
        stages=[col[0] for col in one_hot_pipelines] + [assembler, scaler, model]
    )

    paramGrid = (
        ParamGridBuilder()
        .addGrid(model.maxDepth, [5, 10, 15])
        .addGrid(model.maxBins, [16, 64])
        .addGrid(model.stepSize, [0.05, 0.1])
        .addGrid(model.maxIter, [10, 50])
        .build()
    )

    return TrainValidationSplit(
        estimator=pipeline,
        estimatorParamMaps=paramGrid,
        evaluator=RegressionEvaluator(),
        trainRatio=0.7,
    )

In [36]:
model = get_gbt_regresion_pipeline(continuous_cols, categorica_cols, label)

In [37]:
model = model.fit(train_df)

In [38]:
model.write().overwrite().save('./models/gbt2')

In [39]:
prediction_df = model.transform(test_df).select("features", "label", "prediction").checkpoint()

In [40]:
evaluator_rmse = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="rmse"
)

evaluator_mae = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="mae"
)

evaluator_r2 = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="r2"
)

In [41]:
rmse = evaluator_rmse.evaluate(prediction_df)
mae = evaluator_mae.evaluate(prediction_df)
r2 = evaluator_r2.evaluate(prediction_df)

In [42]:
print(f"RMSE: {rmse}, MAE: {mae}, R^2: {r2}")

RMSE: 14.189673242940854, MAE: 2.6733151733503244, R^2: 0.9125289425323351


## Análisis de resultados

5. Análisis de resultados. Para esta última etapa del proceso, analizarás en profundidad los resultados obtenidos a partir del proceso de entrenamiento implementado. Se deberá de crear una sección titulada “5 Análisis de resultados”, en la cual incluya una profunda reflexión de los resultados alcanzados, identificando sus fortalezas y áreas de oportunidad de los resultados obtenidos.

En este análisis se ha evaluado el rendimiento de diferentes modelos de regresión aplicados para predecir los retrasos en la llegada de vuelos, utilizando métricas como RMSE, MAE y R².

**Fortalezas:**
- **Interpretabilidad de las métricas:** Las medidas de error (RMSE y MAE) proporcionan una comprensión clara de la magnitud de los errores en minutos, lo que es crucial para un análisis aplicado en el ámbito aeroportuario.
- **Captura de relaciones complejas:** La implementación de modelos tanto lineales como de Gradient-Boosted Trees ha permitido capturar tanto relaciones lineales como no lineales en los datos, adaptándose a la distribución sesgada y a la presencia de valores atípicos.
- **Representatividad de los datos:** La cuidadosa construcción de la muestra M y la subsiguiente partición estratificada para entrenamiento y prueba han contribuido a minimizar sesgos en la estimación del rendimiento de los modelos.
- **Utilización de técnicas de escalado y codificación:** El uso de pipelines para One-Hot Encoding y escalado de características ha facilitado el procesamiento consistente y reproducible de las variables, lo que ayuda a estabilizar el comportamiento de los modelos.

**Áreas de oportunidad:**
- **Manejo de valores atípicos:** Dada la fuerte asimetría y la presencia de colas pesadas en la variable objetivo, existe la posibilidad de explorar técnicas de transformación (como logaritmos) o modelos robustos que mitiguen el efecto de los valores extremos.
- **Optimización de hiperparámetros:** Si bien se ha realizado una búsqueda de hiperparámetros, la implementación de estrategias más exhaustivas o el uso de validación cruzada más profunda podría mejorar aún más el desempeño de los modelos.
- **Ampliación del conjunto de modelos:** La exploración de otros algoritmos, incluyendo enfoques de aprendizaje profundo, podría ofrecer mejoras adicionales y capturar de forma más precisa patrones complejos presentes en los datos.
- **Análisis de errores y visualización:** Un análisis más detallado de los errores de predicción y la incorporación de técnicas de visualización adicionales permitirían una interpretación más rica de los resultados, facilitando la comunicación a stakeholders no técnicos.

En conclusión, los modelos desarrollados muestran un desempeño prometedor en la predicción de retrasos, sustentado por un conjunto significativo de métricas. Sin embargo, los retos derivados de la distribución de la variable actividad sugieren que explorar estrategias adicionales de preprocesamiento, validación y selección de modelos podría conducir a soluciones aún más robustas y precisas.